In [1]:
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [2]:
import pandas as pd
import jinja2


# Path to your downloaded CSV file
csv_file_path = "/Users/artashesmatshkalyan/Downloads/WeatherEvents_Jan2016-Dec2022.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

In [3]:
df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8627181 entries, 0 to 8627180
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   EventId            object 
 1   Type               object 
 2   Severity           object 
 3   StartTime(UTC)     object 
 4   EndTime(UTC)       object 
 5   Precipitation(in)  float64
 6   TimeZone           object 
 7   AirportCode        object 
 8   LocationLat        float64
 9   LocationLng        float64
 10  City               object 
 11  County             object 
 12  State              object 
 13  ZipCode            float64
dtypes: float64(4), object(10)
memory usage: 921.5+ MB


In [5]:
# Step 1: Convert 'StartTime(UTC)' column to datetime format
df['StartTime(UTC)'] = pd.to_datetime(df['StartTime(UTC)'])

# Step 2: Extract year from 'StartTime(UTC)' column
df['Year'] = df['StartTime(UTC)'].dt.year

# Step 3: Create the pivot table
pivot_df = df.pivot_table(index='State', columns='Year', values='Precipitation(in)', aggfunc='sum')

# Sort the columns in ascending order
pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)

# Display the pivot table
print(pivot_df)

Year       2016      2017      2018     2019     2020     2021     2022
State                                                                  
AL      3115.76   4177.86   3912.48  3608.45  4229.70  4200.09  3286.27
AR      9124.29   2751.58   3409.64  3797.28  3351.14  2899.97  4845.02
AZ      1118.57    531.17    536.22   451.01   204.26   493.53   452.47
CA      2821.14   2801.65   1966.09  3500.92  1524.39  2323.84  1479.94
CO      1003.03    881.52    689.62   771.04   518.86   803.08   787.76
CT       559.75    610.53    943.96   855.07   667.14   799.79   635.59
DE       247.00    242.59    316.40   286.60   276.06   184.35   191.39
FL      8750.89   9530.59   7844.88  7384.28  7922.32  7403.73  7057.80
GA      1803.96   2471.31   3201.78  2706.03  3536.19  3531.01  2694.45
IA      2887.96   2118.78   2946.21  2479.26  1686.59  1728.10  1632.66
ID       696.24    508.69    482.62   458.79   448.71   427.57   386.13
IL      5413.21   3185.68   3928.99  3968.97  3272.64  3029.41  

In [6]:
# Initialize the Dash app
app = dash.Dash(__name__)
# Convert 'StartTime(UTC)' column to datetime format
df['StartTime(UTC)'] = pd.to_datetime(df['StartTime(UTC)'])

# Create the layout of the dashboard
app.layout = html.Div([
    html.H1("US Precipitation Dashboard"),
    html.Label("Select Year:"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(df['StartTime(UTC)'].dt.year.unique(), reverse=True)],
        value=2016
    ),
    dcc.Graph(id='choropleth-map')
])


In [7]:
# Define callback to update the choropleth map based on selected year
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_choropleth_map(selected_year):
    # Convert selected year to string
    selected_year_str = str(selected_year)
    
    # Filter dataframe by selected year
    filtered_df = df[df['StartTime(UTC)'].dt.year.astype(str).str.startswith(selected_year_str)]
    
    # Group by state and calculate sum of precipitation
    state_precipitation = filtered_df.groupby('State')['Precipitation(in)'].sum().reset_index()
    
    # Merge state precipitation data with US state boundaries data
    fig = px.choropleth(locations=state_precipitation['State'],
                        locationmode="USA-states",
                        color=state_precipitation['Precipitation(in)'],
                        scope="usa",
                        color_continuous_scale="Viridis",
                        range_color=(0, state_precipitation['Precipitation(in)'].max()),
                        labels={'color': 'Precipitation (in)'},
                        title=f"US Precipitation in {selected_year}")
    
    fig.update_layout(geo=dict(bgcolor='rgba(0,0,0,0)',
                               lakecolor='rgba(0,0,0,0)'))
    
    return fig


In [15]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

# Assuming df is your existing DataFrame

# Convert 'StartTime(UTC)' column to string format
df['StartTime(UTC)'] = df['StartTime(UTC)'].astype(str)

# Create the layout of the dashboard
app.layout = html.Div([
    html.Div([
        html.H1("US Precipitation Dashboard", 
                style={'textAlign': 'center',
                       'color': '#007BFF',  # Blue color
                       'fontFamily': 'Arial, sans-serif',
                       'fontSize': '36px',
                       'marginBottom': '10px',  # Add space below the title
                       'textShadow': '2px 2px 4px #000000',  # Add text shadow for depth
                       'cursor': 'pointer',  # Change cursor on hover
                       'transition': 'transform 0.3s ease-in-out',  # Add smooth transition on hover
                       'transformOrigin': '50% 50%'}),
    ], style={'padding': '20px',  # Add padding around the title
              'backgroundColor': '#f8f9fa',  # Light gray background
              'borderBottom': '2px solid #007BFF',  # Blue border at the bottom
              'borderRadius': '5px',  # Rounded corners
              'boxShadow': '2px 2px 4px #888888'}),  # Add box shadow for depth
    html.Label("Select Year:", style={'margin-left': '10px', 'color': '#007BFF', 'fontFamily': 'Arial, sans-serif'}),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(df['StartTime(UTC)'].str[:4].unique(), reverse=True)],
        value='2016',
        style={'width': '50%', 'margin-left': '10px'}
    ),
    dcc.Graph(id='choropleth-map')
], style={'width': '80%', 'margin': 'auto'})

# Define callback to update the choropleth map based on selected year
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_choropleth_map(selected_year):
    # Filter dataframe by selected year
    filtered_df = df[df['StartTime(UTC)'].str.startswith(selected_year)]
    
    # Group by state and calculate sum of precipitation
    state_precipitation = filtered_df.groupby('State')['Precipitation(in)'].sum().reset_index()
    
    # Merge state precipitation data with US state boundaries data
    fig = px.choropleth(locations=state_precipitation['State'],
                        locationmode="USA-states",
                        color=state_precipitation['Precipitation(in)'],
                        scope="usa",
                        color_continuous_scale="Viridis",
                        range_color=(0, state_precipitation['Precipitation(in)'].max()),
                        labels={'color': 'Total Precipitation (in)'},
                        title=f"US Precipitation in {selected_year}")
    
    fig.update_layout(geo=dict(bgcolor='rgba(0,0,0,0)',
                               lakecolor='rgba(0,0,0,0)'),
                      margin=dict(l=0, r=0, t=50, b=0))
    
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
